# Merge American gut data
> Scripts to process every OTU table I have and merge with metadata

In [1]:
# !pip install biom-format
# !pip install anndata
# !pip install scikit-bio
# !pip install tqdm
# !pip install biopython

In [2]:
import biom
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm
import anndata
# from skbio import TreeNode  # For tree
from Bio import SeqIO


In [3]:
metadata = pd.read_table(
    "/home/phil/DATA/americangut/qiita_metadata.txt", 
    dtype={0: str},
    na_values=["not applicable", "missing", "unspecified", "unknown", "NaN", "nan", "N/A", "n/a", "LabControl test", "labcontrol test"],
)
metadata = metadata.set_index(metadata.columns[0])

# Clean up metadata: coerce mixed types to string
for col in metadata.columns:
    if metadata[col].dtype == np.dtype("O"):
        metadata[col] = metadata[col].astype(str)

metadata.head()


/tmp/ipykernel_804250/1428360415.py:1: DtypeWarning: Columns (1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,47,48,49,50,51,52,53,54,55,56,57,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,119,120,121,122,123,124,125,126,127,129,130,131,132,133,134,135,136,138,139,140,141,142,143,144,145,146,147,148,150,151,152,153,154,155,156,157,158,159,168,169,170,171,173,174,175,176,177,178,179,180,181,182,183,184,185,186,188,189,190,191,192,193,194,195,196,197,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,216,217,218,219,220,221,222,223,225,226,227,228,229,230,231,232,233,234,235,236,237,238,240,241,242,243,244,245,246,247,248,249,251,252,253,254,256,257,258,259,260,261,262,263,264,267,268,269,270,272,273,274,275,276,277,279,282,284,286,287,288,289,290,291,292,293,294,295,29

,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_years,alcohol_consumption,alcohol_frequency,alcohol_types_beercider,alcohol_types_red_wine,...,weekday_sleep_time,weekday_wake_time,weekend_sleep_time,weekend_wake_time,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency
sample_name,,,,,,,,,,,,,,,,,,,,,
10317.000001000,nan,no,no,"diagnosed by a medical professional (doctor, p...",60s,NaN,yes,daily,True,False,...,nan,nan,nan,nan,nan,remained stable,NaN,NaN,never,occasionally (1-2 times/week)
10317.000001001,nan,no,no,nan,50s,NaN,yes,rarely (a few times/month),False,False,...,nan,nan,nan,nan,nan,remained stable,NaN,NaN,nan,nan
10317.000001002,nan,no,no,nan,nan,NaN,yes,regularly (3-5 times/week),False,False,...,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,nan
10317.000001004,nan,no,no,nan,40s,NaN,yes,rarely (a few times/month),False,False,...,nan,nan,nan,nan,nan,remained stable,NaN,NaN,nan,nan
10317.000001008,nan,no,no,nan,60s,NaN,no,never,False,False,...,nan,nan,nan,nan,nan,increased more than 10 pounds,NaN,NaN,nan,nan


In [4]:
# For some reason, using biom.concat fails (non-disjoint index). This loop,
# while inelegant, works - go figure.

# This is pretty slow, however:

# dd = "/home/phil/DATA/americangut/qiita_otus/"
# tables = [biom.load_table(f"{dd}{x}") for x in os.listdir(dd)]

# big_table = tables[0]
# it = tqdm(tables[1:], desc=f"{big_table.shape}")
# for table in it:
#     big_table = big_table.merge(table)
#     it.set_description(f"{big_table.shape}")

# with biom.util.biom_open("full_table.biom", 'w') as f:
#     big_table.to_hdf5(f, "merge_data.ipynb")

big_table = biom.load_table("full_table.biom")


FileNotFoundError: [Errno 2] No such file or directory: 'full_table.biom'

In [ ]:
# # Check that every OTU ID is in Greengenes:

# gg_ids = set(
#     [
#         x.id
#         for x in SeqIO.parse(
#             "/data/phil//greengenes/data/gg_13_5.fasta", "fasta"
#         )
#     ]
# )

# otu_ids = set(big_table.ids(axis="observation"))

# print(len(otu_ids.difference(gg_ids)))
# print(len(gg_ids.intersection(otu_ids)))


In [ ]:
# Now check that every sample ID is in the metadata:

sample_ids = set(big_table.ids(axis="sample"))

missing = sample_ids.difference(set(metadata.index))
print(len(missing))
print(len(sample_ids.intersection(set(metadata.index))))


0
32608


In [ ]:
# Create anndata object

# Add metadata to table

big_table.add_metadata(metadata.to_dict(orient="index"), axis="sample")
adata = big_table.to_anndata()

# # Add taxonomy:
# tax = pd.read_table(
#     "/data/phil/greengenes/data/gg_13_5_taxonomy.txt",
#     dtype={0: str},
#     header=None,
# )
# tax = tax.set_index(tax.columns[0])
# tax = tax.loc[adata.var.index]
# adata.var["tax_str"] = tax
# adata.var[
#     ["kingdom", "phylum", "class", "order", "family", "genus", "species"]
# ] = tax[1].str.split(";", expand=True)

# Normalize, save total counts:
# adata.obs["total_counts"] = adata.X.sum(axis=1)
# adata.X /= adata.obs["total_counts"].values.reshape(-1, 1)

# # Add phylogeny:
# tree = TreeNode.read(
#     "/home/phil/phylosig/greengenes/data/gg_13_5_otus/trees/97_otus.tree"
# )
# tree = tree.shear(adata.var.index)
# adata.var["phylogeny"] = tree

# # Add normalized data:
# adata.layers["norm_counts"] = adata.X / adata.X.sum(axis=1, keepdims=True)


In [ ]:
adata.write("data/full_table.h5ad")

In [ ]:
# Save big_table:
# big_table.to_tsv("full_table.tsv.gz", "merge_data.ipynb")

KeyboardInterrupt: 

In [ ]:
adata = big_table.to_anndata()

In [ ]:
# Save adata to file:
# adata.write_h5ad("full_table.h5ad")


AnnData object with n_obs × n_vars = 32608 × 37215
    obs: 'acid_reflux', 'acne_medication', 'acne_medication_otc', 'add_adhd', 'age_cat', 'age_years', 'alcohol_consumption', 'alcohol_frequency', 'alcohol_types_beercider', 'alcohol_types_red_wine', 'alcohol_types_sour_beers', 'alcohol_types_spiritshard_alcohol', 'alcohol_types_unspecified', 'alcohol_types_white_wine', 'allergic_to_i_have_no_food_allergies_that_i_know_of', 'allergic_to_peanuts', 'allergic_to_shellfish', 'allergic_to_tree_nuts', 'allergic_to_unspecified', 'alzheimers', 'animal_age', 'animal_free_text', 'animal_gender', 'animal_origin', 'animal_type', 'animal_type_free_text', 'anonymized_name', 'antibiotic_history', 'appendix_removed', 'artificial_gi_disorder_types_constipation', 'artificial_gi_disorder_types_diarrhea', 'artificial_gi_disorder_types_soft_stools', 'artificial_gi_disorder_types_stomachache', 'artificial_gi_disorder_types_unspecified', 'artificial_gi_disorders', 'artificial_sweeteners', 'artificial_sweetene

In [ ]:
# Load adata from file:
